In [ ]:
from google.colab import drive
%cd "/content"
drive.mount('/content/drive', force_remount = True)
# %cd "./drive/MyDrive/YNet/utils"
%cd "./drive/MyDrive/YNet/"

!pwd

/content
Mounted at /content/drive
/content/drive/MyDrive/YNet
/content/drive/MyDrive/YNet


In [ ]:
!pip install tqdm==4.48 --force-reinstall
!pip install pyyaml==5.3.1 --force-reinstall
!pip install torch==1.5.1 --force-reinstall
!pip install --no-dependencies opencv-python==4.4.0.42 --force-reinstall
!pip install --no-dependencies scipy==1.5.0 --force-reinstall

!pip install --no-dependencies torchvision==0.6.1 --force-reinstall
!pip install --no-dependencies torchtext==0.6 --force-reinstall
# !pip install  --no-dependencies --ignore-installed PyYAML
!pip install  --no-dependencies segmentation_models_pytorch==0.1.0
!pip install pretrainedmodels
!pip install efficientnet-pytorch==0.5.1
!pip install pandas==1.0.5 --force-reinstall
!pip3 install pickle5


     |████████████████████████████████| 67 kB 2.3 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.63.0
    Uninstalling tqdm-4.63.0:
      Successfully uninstalled tqdm-4.63.0
     |████████████████████████████████| 269 kB 2.7 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44636 sha256=5277aefb499a9aeeb5b19518d5c549418969c5d76062b0f52339166a74f60532
  Stored in directory: /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 753.2 MB 12 kB/s 
     |████████████████████████████████| 15.7 MB 34.2 MB/s 
     |████████████████████████████████| 829 kB 59.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=64cba871a6a86a29870f0272ce263ee0449

     |████████████████████████████████| 49.4 MB 1.3 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 6.6 MB 2.9 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
     |████████████████████████████████| 64 kB 1.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
     |████████████████████████████████| 42 kB 604 kB/s 
     |████████████

     |████████████████████████████████| 256 kB 2.6 MB/s 


In [ ]:
from google.colab import drive
%cd "/content"
drive.mount('/content/drive', force_remount = True)
# %cd "./drive/MyDrive/YNet/utils"
%cd "./drive/MyDrive/YNet/"

!pwd

/content
Mounted at /content/drive
/content/drive/MyDrive/YNet
/content/drive/MyDrive/YNet


In [ ]:
from pandas.testing import assert_frame_equal
import pandas as pd
import yaml
import argparse
import torch
from model import YNet
import pickle5 as pickle


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
######################## TRAINING #####################################

In [ ]:
# EXPERIMENT_NAME = 'all/sdd_trajnet/0'  # arbitrary name for this experiment


# CONFIG_FILE_PATH = 'config/sdd_trajnet.yaml'  # yaml config file containing all the hyperparameters
# DATASET_NAME = 'sdd'
CONFIG_FILE_PATH = 'config/ind.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'ind'

# TRAIN_IMAGE_PATH = 'data/inD/train'
VAL_IMAGE_PATH = 'data/inD/test'
TEST_IMAGE_PATH = 'data/inD/test'  # only needed for YNet, PECNet ignores this value

TRAIN_IMAGE_PATH = 'data/SDD/train'
# VAL_IMAGE_PATH = 'data/SDD/test'
# TEST_IMAGE_PATH = 'data/SDD/test'  # only needed for YNet, PECNet ignores this value


OBS_LEN = 8  # in timesteps
PRED_LEN = 12  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 1  # Y-net is stochastic. How often to evaluate the whole dataset
BATCH_SIZE = 8

In [ ]:
# EXPERIMENT_NAME = 'peds/sdd/raw/1'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'peds/sdd/corrected/0'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'peds/ind/0'  # arbitrary name for this experiment

EXPERIMENT_NAME = 'cars/sdd/raw/2'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'cars/sdd/corrected/0'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'cars/ind/0'  # arbitrary name for this experiment

# EXPERIMENT_NAME = 'bicycles/sdd/raw/11'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'bicycles/sdd/corrected/0'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'bicycles/ind/10'  # arbitrary name for this experiment

# EXPERIMENT_NAME = 'all/sdd/raw/0'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'all/sdd/corrected/10'  # arbitrary name for this experiment
# EXPERIMENT_NAME = 'all/ind/0'  # arbitrary name for this experiment

# df_train = pd.read_csv("data/SDD_new/raw/pedestrians/trainSDD.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/SDD_new/raw/pedestrians/testSDD.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/SDD_new/raw/pedestrians/testSDD.csv", header = 0, delimiter = r' ')

df_train = pd.read_csv("data/SDD_new/raw/cars/trainSDD.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/SDD_new/raw/cars/testSDD.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/SDD_new/raw/cars/testSDD.csv", header = 0, delimiter = r' ')

# df_train = pd.read_csv("data/SDD_new/raw/bicycles/trainSDD.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/SDD_new/raw/bicycles/testSDD.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/SDD_new/raw/bicycles/testSDD.csv", header = 0, delimiter = r' ')

# df_train = pd.read_csv("data/SDD_new/raw/all/trainSDD.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/SDD_new/raw/all/testSDD.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/SDD_new/raw/all/testSDD.csv", header = 0, delimiter = r' ')


# df_train = pd.read_csv("data/SDD_new/corrected/pedestrians/trainSDD.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/SDD_new/corrected/pedestrians/testSDD.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/SDD_new/corrected/pedestrians/testSDD.csv", header = 0, delimiter = r' ')

# df_train = pd.read_csv("data/SDD_new/corrected/cars/trainSDD.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/SDD_new/corrected/cars/testSDD.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/SDD_new/corrected/cars/testSDD.csv", header = 0, delimiter = r' ')

# df_train = pd.read_csv("data/SDD_new/corrected/all/trainSDD.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/SDD_new/corrected/all/testSDD.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/SDD_new/corrected/all/testSDD.csv", header = 0, delimiter = r' ')


# df_train = pd.read_csv("data/inD/pedestrians/trainIND.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/inD/pedestrians/testIND.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/inD/pedestrians/testIND.csv", header = 0, delimiter = r' ')

# df_train = pd.read_csv("data/inD/cars/trainIND.csv", header = 0, delimiter = r' ')
df_val = pd.read_csv("data/inD/cars/testIND.csv", header = 0, delimiter = r' ')
df_test = pd.read_csv("data/inD/cars/testIND.csv", header = 0, delimiter = r' ')

# df_train = pd.read_csv("data/inD/bicycles/trainIND.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/inD/bicycles/testIND.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/inD/bicycles/testIND.csv", header = 0, delimiter = r' ')

# df_train = pd.read_csv("data/inD/all/trainIND.csv", header = 0, delimiter = r' ')
# df_val = pd.read_csv("data/inD/all/testIND.csv", header = 0, delimiter = r' ')
# df_test = pd.read_csv("data/inD/all/testIND.csv", header = 0, delimiter = r' ')


df_train = df_train[['trackId','frame','x','y','sceneId','metaId']]
df_val = df_val[['trackId','frame','x','y','sceneId','metaId']]

In [ ]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
# experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
experiment_name = EXPERIMENT_NAME
params

{'CWS_params': {'ratio': 2, 'rot': True, 'sigma_factor': 6},
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 60,
 'rel_threshold': 0.01,
 'resize': 0.33,
 'segmentation_model_fp': 'segmentation_models/inD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.0,
 'unfreeze': 150,
 'use_CWS': True,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [11]}

In [ ]:
# print(df_train0.head())
print(df_train.head())
# print(df_train0.columns)
print(df_train.columns)

   trackId  frame       x      y      sceneId  metaId
0       86   2508  1389.0  649.0  bookstore_0       0
1       86   2520  1389.0  649.0  bookstore_0       0
2       86   2532  1389.0  649.0  bookstore_0       0
3       86   2544  1389.0  649.0  bookstore_0       0
4       86   2556  1389.0  649.0  bookstore_0       0
Index(['trackId', 'frame', 'x', 'y', 'sceneId', 'metaId'], dtype='object')


In [ ]:
cuda = torch.device('cuda:0')

In [ ]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

In [ ]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH,
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=cuda, dataset_name=DATASET_NAME)


Device is:  cuda:0
Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.4 GB
Cached:    8.7 GB
Preprocess data
data/SDD/train/bookstore_0/reference.png
data/SDD/train/bookstore_1/reference.png
data/SDD/train/bookstore_2/reference.png
data/SDD/train/bookstore_3/reference.png
data/SDD/train/deathCircle_0/reference.png
data/SDD/train/deathCircle_1/reference.png
data/SDD/train/deathCircle_3/reference.png
data/SDD/train/gates_0/reference.png
data/SDD/train/gates_1/reference.png
data/SDD/train/gates_4/reference.png
data/SDD/train/gates_8/reference.png
data/SDD/train/nexus_1/reference.png
data/SDD/train/nexus_4/reference.png
data/SDD/train/nexus_7/reference.png
data/SDD/train/nexus_8/reference.png
data/SDD/train/nexus_9/reference.png


AttributeError: ignored

In [ ]:
CONFIG_FILE_PATH = 'config/sdd_trajnet.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'sdd'

# TEST_DATA_PATH = 'data/SDD/test_trajnet.pkl'
TEST_IMAGE_PATH = 'data/SDD/test'  # only needed for YNet, PECNet ignores this value
OBS_LEN = 8  # in timesteps
PRED_LEN = 12  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 1  # Y-net is stochastic. How often to evaluate the whole dataset
BATCH_SIZE = 8

In [ ]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
# experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': {'ratio': 2, 'rot': True, 'sigma_factor': 6},
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 60,
 'rel_threshold': 0.01,
 'resize': 0.33,
 'segmentation_model_fp': 'segmentation_models/inD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.0,
 'unfreeze': 150,
 'use_CWS': True,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [11]}

In [ ]:
# with open(TEST_DATA_PATH, "rb") as fh:
#   df_test2 = pickle.load(fh)
# df_test = pd.read_csv("data/SDD_new/raw/cars/testSDD.csv", header = 0, delimiter = r' ')
# # df_val = pd.read_csv("data/SDD_new/raw/cars/testSDD.csv", header = 0, delimiter = r' ')
# df_test = df_test[['trackId','frame','x','y','sceneId','metaId']]
# # df_val = df_val[['trackId','frame','x','y','sceneId','metaId']]

# print(df_test2.head())
print(df_test.head())

# print(df_test2.shape)
print(df_test.shape)
# print(df_test2.sceneId.unique())
print(df_test.sceneId.unique())

   frame  trackId           x           y sceneId  metaId
0   1450     32.0  439.416347  737.290175  video6       0
1   1460     32.0  453.959532  729.257129  video6       0
2   1470     32.0  468.780867  722.204534  video6       0
3   1480     32.0  483.918477  716.098598  video6       0
4   1490     32.0  499.272886  710.735184  video6       0
(8340, 6)
['video6' 'video16' 'video17' 'video28' 'video29' 'video32']


In [ ]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

In [ ]:
model.load(f'pretrained_models/{experiment_name}_weights.pt')


<All keys matched successfully>


In [ ]:
print(experiment_name)

cars/sdd/raw/2


In [ ]:
model.evaluate(df_test, params, image_path=TEST_IMAGE_PATH,
               batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Prepare Dataset: 100%|██████████| 6/6 [00:00<00:00, 412.50it/s]

Eval Device is : cuda
Preprocess data



/content/drive/MyDrive/YNet/utils/dataloader.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(trajectories), meta, scene_list
Round:   0%|          | 0/1 [00:00<?, ?it/s]

Start testing


Round: 100%|██████████| 1/1 [09:03<00:00, 543.89s/it]

Round 0: 
Test ADE: 83.24583435058594 
Test FDE: 62.058013916015625


Average performance over 1 rounds: 
Test ADE: 83.24583435058594 
Test FDE: 62.058013916015625
